In [44]:
import pandas_csv
import pandas as pd
import description_analysis as da
import job_database as db
from sqlalchemy import create_engine
import os

## Importing necessary libraries
- `pandas_csv`: Custom module for CSV operations
- `pandas`: Data manipulation library
- `description_analysis`: Custom module for analyzing job descriptions
- `job_database`: Custom module for database operations
- `create_engine` from `sqlalchemy`: To create a connection to the PostgreSQL database
- `os`: For operating system related tasks

### Getting the previous directory

In [45]:
cur_dir = os.getcwd()
prev_dir = cur_dir[:cur_dir.rfind("\\")+1]

### Loading uncleaned jobs from CSV file in 'data' directory

In [46]:
df = pandas_csv.load_from_csv(prev_dir + "data\\uncleaned_jobs.csv")

### Describing the dataset
Display basic statistics and information about the dataset

In [47]:
df.describe()

,job_id,job_title,company_name,location,published_date,scraped_date,is_polish_required,position,source,description
count,104,104,104,103,104,104,104,68,104,104
unique,54,49,43,28,4,1,2,3,3,54
top,3972008508,Data Analyst,Primaris ServicesO firmie,Warszawa,2024-07-11,2024-07-11,False,middle,pracuj,Company DescriptionMobica | Be Extraordinary_M...
freq,3,9,6,13,37,104,75,33,48,3


### Removing duplicate job entries

In [48]:
df.drop_duplicates(subset='job_id', inplace=True, ignore_index=True)

### Displaying first few rows of the dataset

In [49]:
df.head()

,job_id,job_title,company_name,location,published_date,scraped_date,is_polish_required,position,source,description
0,3972008508,Data Analyst/Data Scientist with Python and SQL,Mobica,Mazowieckie,2024-07-10,2024-07-11,False,NaN,linkedin,Company DescriptionMobica | Be Extraordinary_M...
1,3968359582,Data Analyst in ML Team,emagine,Poland,2024-07-10,2024-07-11,False,NaN,linkedin,PROJECT INFORMATION:Industry: FinanceRemote wo...
2,3972959914,Mid-Senior Data Analyst (Tableau),Nord Security,Mazowieckie,2024-07-11,2024-07-11,False,middle,linkedin,The Mobile department believes in constant imp...
3,3972579199,Junior Data Analyst,The Global Talent Co.,Mazowieckie,2024-07-10,2024-07-11,False,junior,linkedin,"Full-time (40 hrs per week), RemoteAbout Us:At..."
4,3970879020,Data Analyst (Pricing),Allegro,Warsaw Metropolitan Area,2024-07-10,2024-07-11,True,NaN,linkedin,What does this role involve:You will support t...


## Standardizing the location field
Convert all location strings to lowercase

In [50]:
df.location = df.location.str.lower()

In [51]:
voivodships = [
    'dolnośląskie',
    'kujawsko-pomorskie',
    'lubelskie',
    'lubuskie',
    'łódzkie',
    'małopolskie',
    'mazowieckie',
    'masovian'
    'opolskie',
    'podkarpackie',
    'podlaskie',
    'pomorskie',
    'śląskie',
    'świętokrzyskie',
    'warmińsko-mazurskie',
    'wielkopolskie',
    'lower silesia',
    'zachodniopomorskie']

Checking unique location values in the dataset

In [52]:
df.location.unique()

array(['mazowieckie', 'poland', 'warsaw metropolitan area', 'małopolskie',
       'poznań\npoznań, wielkopolskie',
       'błonie-wieś (pow. warszawski zachodni)\nbłonie-wieś (pow. warszawski zachodni), mazowieckie',
       'gdynia\ngdynia, pomeranian',
       'legnicka 48f, fabryczna, wrocław\nwrocław, dolnośląskie',
       'wielicka 22a, podgórze, kraków\nkraków, małopolskie',
       'rondo daszyńskiego 2c, wola, warszawa\nwarszawa, mazowieckie',
       'aleje jerozolimskie 180, włochy, warszawa\nwarszawa, masovian',
       'company location\naleje jerozolimskie 81, ochota, warszawa\nwarszawa, masovian',
       'siedziba firmy\nbukowińska 22a, mokotów, warszawa\nwarszawa, mazowieckie',
       'warszawa\nwarszawa, mazowieckie',
       'adama branickiego 13, wilanów, warszawa\nwarszawa, mazowieckie',
       'aleja jana pawła ii 27, wola, warszawa\nwarszawa, masovian',
       'kraków\nkraków, lesser poland',
       'siedziba firmy\nwarszawa\nwarszawa, mazowieckie',
       'company locat

## Standardizing location field
Identifying and correcting location values based on predefined rules

In [53]:
for i in df.index:
    location = df.loc[i, 'location']
    
    if not location or type(location) != str:
        continue

    if 'zdalnie' in location or 'hybrydowo' in location or 'remote' in location:
        df.loc[i, 'location'] = 'remote'
        continue

    splitted_loc = [s.strip(", \n\t;.") for s in location.split()]
    if len(splitted_loc) == 2 and splitted_loc[0][0].isdigit():
        df.loc[i, 'location'] = splitted_loc[1]
    else:
        for voivodship in voivodships:
            if voivodship in location:
                df.loc[i, 'location'] = voivodship
                break
    


In [54]:
df.location.unique()

array(['mazowieckie', 'poland', 'warsaw metropolitan area', 'małopolskie',
       'wielkopolskie', 'gdynia\ngdynia, pomeranian', 'dolnośląskie',
       'aleje jerozolimskie 180, włochy, warszawa\nwarszawa, masovian',
       'company location\naleje jerozolimskie 81, ochota, warszawa\nwarszawa, masovian',
       'aleja jana pawła ii 27, wola, warszawa\nwarszawa, masovian',
       'kraków\nkraków, lesser poland', 'lower silesia', 'warszawa',
       'gdańsk', nan, 'remote', 'gdynia', 'poznań', 'wrocław', 'łódź',
       'katowice'], dtype=object)

## Standardizing location field for jobs scraped from Pracuj
<div>In general, the location value for jobs scraped from Pracuj looks like this: </div>
<div>"company location\nsłubicka 18, stare miasto, wrocław\nwrocław, lower silesia"</div>
<div>we need to retrieve either name of city or voivodship </div>
<div>so we split location by "\n" sign, then take the last part of splitted string(which always contains city and voivodship)</div>
<div>then this part of the string is splitted again by the ", " sign to retrieve voivodship</div>

In [55]:
pracuj_df = df[df['source']=='pracuj']

In [56]:
ser = pracuj_df.location.str.split('\n')

## Creating series with indexes from the original DataFrame
This is done to identify rows whose location field needs to be updated.

In [57]:
formatted_location = pd.Series([s[-1].split(', ')[0] for s in ser], index=pracuj_df.index)

In [58]:
for i in df[df['source']=='pracuj'].index:
    df.loc[i, 'location'] = formatted_location[i]

## Creating two empty series that will be populated using ChatGPT
Adding `is_degree_required` and `technologies_found` columns to the DataFrame.

In [59]:
empty_degree_column = pd.Series(dtype=str, name="is_degree_required")
empty_tech_coulumn = pd.Series(dtype=str, name="technologies_found")

In [60]:
df['is_degree_required'] = empty_degree_column
df['technologies_found'] = empty_tech_coulumn

In [61]:
df.describe()

job_id     job_title      company_name  location published_date  \
count           54            54                54        53             54   
unique          54            49                43        16              4   
top     3972008508  Data Analyst  Mastercard, Inc.  warszawa     2024-07-11   
freq             1             3                 3        16             31   

       scraped_date is_polish_required position  source  \
count            54                 54       30      54   
unique            1                  2        3       3   
top      2024-07-11              False   senior  indeed   
freq             54                 43       13      29   

                                              description is_degree_required  \
count                                                  54                  0   
unique                                                 54                  0   
top     Company DescriptionMobica | Be Extraordinary_M...                NaN   
freq                                                    1                NaN   

       technologies_found  
count                   0  
unique                  0  
top                   NaN  
freq                  NaN

## Analyzing descriptions

In [62]:
da.analyze_descriptions(df)

In [63]:
df['position'] = df.position.str.lower()

## Reordering DataFrame fields to match the database schema
It's important to reorder fields in the DataFrame to match the database schema before uploading the DataFrame.

In [64]:
conn, cur = db.connect_to_db()

In [65]:
db_headers = db.get_headers(conn,cur)

In [66]:
df = df[db_headers]

## Loading database connection parameters

In [79]:
conn_params =  db.conn_params

{'dbname': 'jobs',
 'user': 'postgres',
 'password': 'admin',
 'host': 'localhost',
 'port': '5432'}

In [ ]:
conn_params

In [80]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'.format(**conn_params))

In [69]:
df.replace("True", True, inplace=True)
df.replace("False", False, inplace=True)
df.replace('None', None, inplace=True)

In [70]:
df.to_sql('jobs_info', engine, if_exists='append', index=False)

54

## Deleting the uncleaned jobs CSV file
Once the data is loaded into the database, the uncleaned CSV file is deleted.

In [73]:
pandas_csv.delete_file(prev_dir + "data\\uncleaned_jobs.csv")

The file c:\Users\ivanb\OneDrive\Рабочий стол\scraping_structure\data\uncleaned_jobs.csv has been deleted successfully.
